## comp_mean_grid
Compute grid-related variables from time-average data (SSH), including corrections for vertical grid metrics

Namely:
* corrected e3t and e3w
* hbot (bottom depth)

Strategy: first copy all vars in zarr file, then compute correction reading from zarr file (simpler)

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

from pathlib import Path
import os, time
import numpy as np

import xarray as xr
from xorca.lib import load_xorca_dataset
import itidenatl.gridop as gop
import itidenatl.utils as ut

In [2]:
from dask.distributed import Client, LocalCluster, wait
cluster = LocalCluster(threads_per_worker=8) #n_workers=24, threads_per_worker=1, memory_limit=8e6,silence_logs=50
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35597 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 56 Memory: 236.33 GiB


In [7]:
scratch = Path(os.getenv("SCRATCHDIR"))
workdir = Path("/work/CT1/ige2071/nlahaye")

grid_path = scratch #Path("/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/")
mean_path = Path("/work/CT1/ige2071/SHARED/mean")

ssh_fname = "global_mean_gridT-2D.zarr"
grid_fname = "mesh_mask_eNATL60_3.6.nc"

out_file = scratch/"eNATL60_mean_grid_z.zarr"

grid_files = [grid_path/grid_fname]

In [8]:
%%time
chk_xy, chk_z = 1200, 10
chunks = {dim:chk_xy for dim in ["x_c", "x_r", "y_c", "y_r"]}
chunks.update({dim:chk_z for dim in ["z_c", "z_l"]})

ds_tot = load_xorca_dataset(data_files=[], aux_files=grid_files,
                              decode_cf=True, model_config="nest", target_ds_chunks=chunks
                             )
print("initial dataset is {:.1f} GB".format(ds_tot.nbytes/1e9))
ds_new = ds_tot.drop_dims(("y_r", "x_r"))\
                .reset_coords(names=("e1t", "e2t"), drop=True)
print("working dataset is {:.1f} GB".format(ds_new.nbytes/1e9))

varnames = [var.name for var in ds_new.coords.values() if len(var.dims)==3]
print("keeping variables", varnames)
ds_new = ds_new.reset_coords(varnames)

ds_new

initial dataset is 525.4 GB
working dataset is 296.7 GB
keeping variables ['depth_c_3d', 'depth_l_3d', 'e3t', 'e3w', 'tmask']
CPU times: user 15.9 s, sys: 1.47 s, total: 17.4 s
Wall time: 17.1 s


<xarray.Dataset>
Dimensions:     (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 ... 293 294 295 296 297 298 299 300
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
  * y_c         (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
  * x_c         (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
    depth_c     (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l     (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil   (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
Data variables:
    depth_c_3d  (z_c, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    depth_l_3d  (z_l, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3t         (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w         (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask       (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>

### create zarr archive and copy targeted data
Warning, this will not store coordinates (except dimension coordinates). Storing coordinates will be done as a second step

In [6]:
%%time
### open zarr archive. this may take a minute or so
ds_new.to_zarr(out_file, mode="w", compute=False, consolidated=True)

Delayed('_finalize_store-128ffd36-a05c-4662-8a69-eb0bce2e4802')

In [7]:
def dist_write(ds, varname, store_dir, dim="z"):
    """ utilitary function that loops over a dimension chunk to store in a zarr archive """
    sds = ds.get([varname])
    dim = next(di for di in sds.dims if di.startswith(dim))
    co_vir = [co for co in sds.coords]
    sds = sds.drop([co for co in sds.coords])
    chk_z = np.r_[0, np.array(sds.chunks[dim]).cumsum()]
    print("now doing {} chunk (.../{}):".format(dim,chk_z.size-1), end=" ")
    for iz in range(chk_z.size-1):
        print(iz, end=", ")
        sliz = slice(chk_z[iz], chk_z[iz+1])
        sds.isel({dim:sliz}).to_zarr(store_dir, mode="a", region={dim:sliz})
    return sds

In [8]:
%%time
### loop over variables (and z_chunks internally)
tmei = time.time()
for lavar in varnames:
    print("variable", lavar, end="; ")
    tmeb = time.time()
    dist_write(ds_new, lavar, out_file, dim="z")
    print("timing for {}: wall {:.0f}".format(lavar,time.time()-tmeb))
print("total time: {:.0f}'{:.0f}''".format((time.time()-tmei)//60, (time.time()-tmei)%60))

variable depth_c_3d; now doing z_c chunk (.../30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, timing for depth_c_3d: wall 409
variable depth_l_3d; now doing z_l chunk (.../30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, timing for depth_l_3d: wall 413
variable e3t; now doing z_c chunk (.../30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, timing for e3t: wall 463
variable e3w; now doing z_l chunk (.../30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, timing for e3w: wall 451
variable tmask; now doing z_c chunk (.../30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, timing for tmask: wall 430
total time: 36'5''
CPU times: user 27min 1s, sys: 1min 39s, total: 28min 

## load SSH, compute hbot and store
this will also store the coordinates

In [15]:
### load and merge SSH, compute hbot
ds_re = xr.open_zarr(out_file)

### copy non-dimension coordinates
for co in ds_re.coords:
    if co not in ds_re.dims:
        ds_re[co] = ds_new[co]
        
dims = {"ssh": ("x_c", "y_c"), "temp":("x_c", "y_c", "z_c"), "salt":("x_c", "y_c", "z_c")}
dimap = {"x":"x_c", "y":"y_c", "deptht":"z_c"}
chk_tot = ds_re.chunks

def get_var(path, varname):
    da = xr.open_zarr(path)[varname].squeeze()
    da = da.rename({dim:dimap[dim] for dim in da.dims})
    da = da.chunk({dim:chk_tot[dim] for dim in da.dims})
    return da

ds_re["sossheig"] = get_var(mean_path/ssh_fname, "sossheig").persist()

ds_re.sossheig.encoding.pop("chunks") # otherwise writing zarr raise error

#ds_re

<xarray.Dataset>
Dimensions:     (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * x_c         (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
  * y_c         (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
    depth_c     (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l     (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil   (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 ... 293 294 295 296 297 298 299 300
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
Data variables:
    depth_c_3d  (z_c, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    depth_l_3d  (z_l, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3t         (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w         (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask       (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    sossheig    (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>

In [21]:
%%time
### compute hbot
ds_re["hbot"] = gop.get_hbot(ds_re, overwrite=True).persist() #ds_re.e3t.where(ds_re.tmask).sum("z_c").persist() #
#ds_re

CPU times: user 3.29 s, sys: 37.1 ms, total: 3.32 s
Wall time: 4.18 s


<xarray.Dataset>
Dimensions:     (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
  * x_c         (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
  * y_c         (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
    depth_c     (z_c) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    depth_l     (z_l) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    llat_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    llon_cc     (y_c, x_c) float32 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    tmaskutil   (y_c, x_c) int8 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 ... 293 294 295 296 297 298 299 300
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
Data variables:
    depth_c_3d  (z_c, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    depth_l_3d  (z_l, y_c, x_c) float32 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3t         (z_c, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    e3w         (z_l, y_c, x_c) float64 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    tmask       (z_c, y_c, x_c) int8 dask.array<chunksize=(10, 1200, 1200), meta=np.ndarray>
    sossheig    (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>
    hbot        (y_c, x_c) float64 dask.array<chunksize=(1200, 1200), meta=np.ndarray>

In [33]:
%%time
### store result for SSH and HBOT. This will store the coordinates as well
ds_re.get(["sossheig", "hbot"]).to_zarr(out_file, mode="a")

CPU times: user 5.23 s, sys: 380 ms, total: 5.61 s
Wall time: 7.43 s


## compute corrected grid (depth and metrics) and store
reading from zarr : very fast

In [34]:
%%time
# compute correction for e3t and e3w
# cf gridop.get_rec_e3z
    
lacorr = (1. + ds_re.sossheig / ds_re.hbot)

### compute corrected vertical grid metrics and store in zarr
(ds_re.get(["e3t", "e3w"]) * lacorr).where(ds_re.tmaskutil).to_zarr(out_file, mode="a")

In [41]:
%%time
### compute corrected 3D depth grids
((ds_re.get(["depth_c_3d","depth_l_3d"])/ds_re.hbot + 1.) * ds_re.sossheig).to_zarr(out_file, mode="a")

CPU times: user 21.3 s, sys: 2.09 s, total: 23.3 s
Wall time: 51.5 s


## post processing
rechunk

In [4]:
ds_re = xr.open_zarr(out_file)
chk_xy, chk_z = 200, 300
new_chks = {d:chk_z for d in ds_re.dims if d.startswith("z")}
new_chks.update({d:chk_xy for d in ["x_c","y_c"]})
ds_re = ds_re.chunk(new_chks)

### remove encoding "chunks" to write with zarr
for k in ds_re.coords.values():
    print(k.name, k.encoding.pop("chunks", None))
for k in ds_re.data_vars.values():
    print(k.name, k.encoding.pop("chunks", None))
    
ds_re

depth_c (10,)
depth_l (10,)
llat_cc (1200, 1200)
llon_cc (1200, 1200)
tmaskutil (1200, 1200)
x_c (8354,)
y_c (4729,)
z_c (300,)
z_l (300,)
depth_c_3d (10, 1200, 1200)
depth_l_3d (10, 1200, 1200)
e3t (10, 1200, 1200)
e3w (10, 1200, 1200)
hbot (1200, 1200)
sossheig (1200, 1200)
tmask (10, 1200, 1200)


<xarray.Dataset>
Dimensions:     (x_c: 8354, y_c: 4729, z_c: 300, z_l: 300)
Coordinates:
    depth_c     (z_c) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    depth_l     (z_l) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    llat_cc     (y_c, x_c) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    llon_cc     (y_c, x_c) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    tmaskutil   (y_c, x_c) int8 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * x_c         (x_c) int64 1 2 3 4 5 6 7 ... 8348 8349 8350 8351 8352 8353 8354
  * y_c         (y_c) int64 1 2 3 4 5 6 7 ... 4723 4724 4725 4726 4727 4728 4729
  * z_c         (z_c) int64 1 2 3 4 5 6 7 8 ... 293 294 295 296 297 298 299 300
  * z_l         (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
Data variables:
    depth_c_3d  (z_c, y_c, x_c) float32 dask.array<chunksize=(300, 200, 200), meta=np.ndarray>
    depth_l_3d  (z_l, y_c, x_c) float32 dask.array<chunksize=(300, 200, 200), meta=np.ndarray>
    e3t         (z_c, y_c, x_c) float64 dask.array<chunksize=(300, 200, 200), meta=np.ndarray>
    e3w         (z_l, y_c, x_c) float64 dask.array<chunksize=(300, 200, 200), meta=np.ndarray>
    hbot        (y_c, x_c) float64 dask.array<chunksize=(200, 200), meta=np.ndarray>
    sossheig    (y_c, x_c) float64 dask.array<chunksize=(200, 200), meta=np.ndarray>
    tmask       (z_c, y_c, x_c) int8 dask.array<chunksize=(300, 200, 200), meta=np.ndarray>

In [5]:
out_new = out_file.parent/(out_file.stem+"_one-chk-z.zarr")
out_new

PosixPath('/scratch/cnt0024/ige2071/nlahaye/eNATL60_mean_grid_z_one-chk-z.zarr')

In [10]:
client.restart()

distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 1 seconds, killing


Client Scheduler: tcp://127.0.0.1:34619 Dashboard: http://127.0.0.1:46806/status,Cluster Workers: 7 Cores: 56 Memory: 236.33 GiB


In [37]:
%%time
ds_re.to_zarr(out_new, mode="w", compute=False, consolidated=True)

CPU times: user 3.61 s, sys: 655 ms, total: 4.27 s
Wall time: 5.44 s


Delayed('_finalize_store-8cd984fe-0fae-422a-9f44-d4ea96e38403')

In [6]:
%%time
ds_re.get([v for v in ds_re.data_vars if len(ds_re[v].dims)<3]).to_zarr(out_new, mode="a")

CPU times: user 7.52 s, sys: 404 ms, total: 7.93 s
Wall time: 9.52 s


In [9]:
%%time
les_var = [v for v in ds_re.data_vars if len(ds_re[v].dims)>=3]
print("copying")
for v in les_var:
    ds_re.get([v]).to_zarr(out_new, mode="a")
    print("v", end=", ")
print("done")

copying
v, v, v, v, v, done
CPU times: user 2min 1s, sys: 8.76 s, total: 2min 10s
Wall time: 4min 9s


## Old cells / Old stuff

In [34]:
%%time
### store variables that do not need modification, or need some computation
var_raw = ["tmask"]
sds = ds_new.get(var_raw)#.isel(z_c=slice(0,10))
sds.to_zarr(workdir/"prov.zarr", mode="w")

CPU times: user 8min 49s, sys: 14.8 s, total: 9min 3s
Wall time: 9min 6s


In [24]:
chk_z = np.r_[0, np.array(ds_new.chunks["z_c"]).cumsum()]
chk_z

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300])

In [33]:
%%time

sds = ds_new.get(["tmask"])
co_vir = [co for co in sds.coords]# if "z_c" not in sds.coords[co].dims]
sds = sds.drop(co_vir)

print("now doing chunk (/{}):".format(chk_z.size-1), end=" ")
for iz in range(chk_z.size-1):
    print(iz, end=", ")
    sliz = slice(chk_z[iz], chk_z[iz+1])
    sds.isel(z_c=sliz).to_zarr(workdir/"prov.zarr", mode="a", region={"z_c":sliz})
print("done")

now doing chunk (/30): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, done
CPU times: user 4min 55s, sys: 19.7 s, total: 5min 15s
Wall time: 7min 12s
